In [ ]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import random

In [ ]:
dic = {
    'source_id':[],
    'manual_id':[],
    'article_title':[],
    'url':[],
    'authors':[],
    'author_affiliation':[],
    'abstract_text':[],
    'date':[],
    'start_time':[],
    'end_time':[],
    'location':[],
    'session_id':[],
    'news_type':[],
    'session_title':[],
    'session_type':[],
    'category':[],
    'sub_category':[],
    'disclosure':[]
    }

In [ ]:
for i in soup.find_all(class_=re.compile('list-group-item loadbyurl.*'))[0:]:
    code = i.get('data-presid')
    print(code)

In [ ]:
c = 0
se_no = 1

for ses in soup.find_all(class_='list-group-item bucket'):
    
    sess_id = f'S{se_no}'
    session_title = ses.find(style='color:#5A5A5A').text.strip()
    print(session_title)
    sess_ke_presentation = ses.next_sibling
    
    for i in sess_ke_presentation.find_all(class_=re.compile('list-group-item loadbyurl.*'))[0:]:
        code = i.get('data-presid')
        icon = ''
        if i.find_all('img'): # check if there is any icon
            print('yesss iconn......')
            for i in i.find_all('img'):
                icon += i.get('title') + '; '
                
        session_url = f'https://gestannual2023.eventscribe.net/fsPopup.asp?PresentationID={code}&mode=presInfo'
        
        req = requests.get(session_url)
        soup = BeautifulSoup(req.content, 'html.parser')
        
        art_title = soup.find('h1').text.strip()
        session_date = soup.find(class_='pull-left pres-tidbit tipsytip').text.strip()
        ses_time_loc = soup.find_all(class_='pull-left pres-tidbit')
        session_time = ses_time_loc[0].text.strip()

        if len(ses_time_loc) > 1:
            session_loc = ses_time_loc[1].text.strip()
        else:
            print('no location....')
            session_loc = ''

        sess_text = ''
        session_author = ''
        affiliation = ''
        sess_type = ''
        category = ''
        cat = soup.find_all(class_='trackname innertracks')
        if len(cat) < 1:
            cat = soup.find_all(style='color:#FFFFFF;font-size:14px; margin-top:10px; margin-right:10px; background:#489CD8')

        try:
            sess_type = soup.find(style='color:#FFFFFF; font-size:14px; margin-top:10px; margin-right:10px; background:#489CD8;').text.strip()
        except:
            try:
                sess_type = soup.find(style='margin-top:10px;').text.strip()
            except:
                sess_type = ''

        for i in cat:
            category += i.text.strip() + '; '

        auth = soup.find(class_='speakers-wrap')

        for s_i in auth.find_all(class_='row'): # author affiliation
            aut = s_i.find(class_='col-xs-12 col-md-6').a.text
            aff = s_i.find(class_="text-muted prof-text")
            aff = re.split('<br.*?>', str(aff))
            aff = ' '.join(aff)
            if re.search('<p class="text-muted prof-text">(.*?)</p>', aff):
                aff = re.search('<p class="text-muted prof-text">(.*?)</p>', aff).group(1)
            else:
                aff = ' '
            session_author += aut.strip() + '; '
            affiliation += aff + '; '

        try: # abs_text with giving spaces
            sess_text = soup.find(class_='PresentationAbstractText mar-top')
            sess_text = str(sess_text)
            sess_text = re.sub('<.*?>', ' ', sess_text)
        except:
            sess_text = ''
        if sess_text == None:
            sess_text = ''

        if art_title == sess_type:
            sess_type = ' '
            print('check if.......')

        category = re.sub(sess_type+'; ', '', category)
        print(f'art title :- {art_title}')
        print(f'session type :- {sess_type}')
        print(f'category :- {category}')
        print(f'session date :- {session_date}')
        print(f'session time :- {session_time}')
        print(f'session location :- {session_loc}')
        print(f'session author :- {session_author}')
        print(f'session aff :- {affiliation}')
        print(f'session text :- {sess_text}')

        print('======================='*5)
        
        if session_title.strip() == art_title:
            dic['news_type'].append('Session')
        else:
            dic['news_type'].append('Abstract')
            
        dic['session_id'].append(sess_id)
        dic['source_id'].append('')
        dic['manual_id'].append('')
        dic['article_title'].append(art_title)
        dic['url'].append(session_url)
        dic['authors'].append(session_author)
        dic['author_affiliation'].append(affiliation)
        dic['abstract_text'].append(sess_text)
        dic['date'].append(session_date)
        dic['start_time'].append(session_time)
        dic['end_time'].append('')
        dic['location'].append(session_loc)
        dic['session_title'].append(session_title)
        dic['session_type'].append(sess_type)
        dic['category'].append(category)
        dic['sub_category'].append('')
        dic['disclosure'].append(icon)
    
    se_no += 1
    c += 1

In [ ]:
df = pd.DataFrame(dic)
df

In [ ]:
df.to_excel('check_event.xlsx', index=False)